In [1]:
# As usual, a bit of setup

import time
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.fc_net import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.solver import Solver

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
import scipy.io as sio
import os
import sys


## Read PIE dataset
def read_mat ( path ):
    c = 0
    X ,y ,isTest= [], [],[]

    for filename in os.listdir ( path ):
        if (filename != 'Thumbs.db'):
            data = sio.loadmat( os.path.join ( path , filename ))
#             np.concatenate((X, data['fea']),axis = 0)
#             np.concatenate((y, data['gnd']),axis = 0)
            if len(X) == 0 and len(y) == 0:
                X = data['fea']
                y = data['gnd']
                isTest = data['isTest']
            else:
                X = np.concatenate((data['fea'], X), axis = 0)
                y =np.concatenate((data['gnd'], y), axis = 0)
                isTest = np.concatenate((data['isTest'],isTest), axis = 0)
    return X, y, isTest

ROOT = 'dataset/PIE dataset/'
X,y,isTest = read_mat(ROOT)
X = np.array(X)
y = np.array(y)
isTest = np.array(isTest)

print X.shape
print X
print y.shape
print y
print isTest.shape
print isTest

(11554, 4096)
[[ 46  52  48 ..., 226 234 238]
 [ 50  39  32 ..., 255 254 254]
 [ 47  47  39 ..., 254 254 254]
 ..., 
 [ 11  10   8 ...,  36  40  42]
 [  9  11   9 ...,  29  31  36]
 [ 12   9   9 ...,  29  34  38]]
(11554, 1)
[[ 1]
 [ 1]
 [ 1]
 ..., 
 [68]
 [68]
 [68]]
(11554, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 1.]
 [ 0.]
 [ 0.]]


In [3]:
X_train, X_test, y_train, y_test = [],[],[],[]
for i in np.arange(len(isTest)):
    if(isTest[i] == 0):
        if len(X_train) == 0 and len(y_train) == 0:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.vstack((X[i], X_train))
            y_train = np.vstack((y[i], y_train))
    else:
        if len(X_test) == 0 and len(y_test) == 0:
            X_test = X[i]
            y_test = y[i]
        else:
            X_test = np.vstack((X[i], X_test))
            y_test = np.vstack((y[i], y_test))

print X_train
print X_train.shape

[[ 12   9   9 ...,  29  34  38]
 [  9  11   9 ...,  29  31  36]
 [  8  10  12 ...,  25  25  22]
 ..., 
 [ 47  47  39 ..., 254 254 254]
 [ 50  39  32 ..., 255 254 254]
 [ 46  52  48 ..., 226 234 238]]
(10262, 4096)


In [13]:
# show the shape of X_train X_test y_train y_test
print X_train.shape
print X_test.shape
X_train = X_train.reshape((X_train.shape[0],64,64))
X_test = X_test.reshape((X_test.shape[0],64,64))
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])
print y_train.shape
print y_test.shape



(10262, 64, 64)
(1292, 64, 64)
(10262,)
(1292,)


In [14]:
num_train = 50
small_data = {
  'X_train': X_train[:num_train],
  'y_train': y_train[:num_train],
  'X_val': X_test,
  'y_val': y_test,
}

learning_rate = 1e-3
weight_scale = 6e-2
model = FullyConnectedNet([64, 64],
                weight_scale=weight_scale, dtype=np.float64)
solver = Solver(model, small_data,
                print_every=10, num_epochs=20, batch_size=25,
                update_rule='sgd',
                optim_config={
                  'learning_rate': learning_rate,
                }
         )
solver.train()

plt.plot(solver.loss_history, 'o')
plt.title('Training loss history')
plt.xlabel('Iteration')
plt.ylabel('Training loss')
plt.show()

ValueError: shapes (25,4096) and (3072,64) not aligned: 4096 (dim 1) != 3072 (dim 0)